In [ ]:
# Lister les fichiers
import os
files = mssparkutils.fs.ls("Files/")
for f in files:
    print(f.name)

In [ ]:
# Vérifier la table landing_validations
spark.sql("SELECT * FROM landing_validations").show()

In [ ]:
# Récupérer les validations qui n'ont pas encore reçu d'email
df_pending_emails = spark.sql("""
    SELECT * FROM landing_validations 
    WHERE is_validated IS NOT NULL 
    AND email_sent_at IS NULL
""").toPandas()

print(f"{len(df_pending_emails)} validations en attente d'email")

# Afficher les données
if len(df_pending_emails) > 0:
    for idx, row in df_pending_emails.iterrows():
        print(f"\nOpportunité: {row['venue_name']}")
        print(f"Email à envoyer à: {row['saler_email']}")
        print(f"Statut: {'OK' if row['is_validated'] == 1 else 'KO'}")

In [ ]:
import requests
import json

token = mssparkutils.credentials.getToken("https://api.fabric.microsoft.com")
workspace_id = spark.conf.get("trident.workspace.id")
pipeline_id = "319746ae-faec-444d-9037-b6d81a386354"

headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}


In [ ]:
df_pending_emails = spark.sql("""
    SELECT * FROM landing_validations 
    WHERE is_validated IS NOT NULL 
    AND email_sent_at IS NULL
""").toPandas()

print(f"{len(df_pending_emails)} validations en attente d'email")

In [ ]:
def format_validation_email(row):
    """Formate le contenu de l'email de notification."""
    status = "OK (Validée)" if row["is_validated"] == 1 else "KO (Rejetée)"
    status_color = "#28a745" if row["is_validated"] == 1 else "#dc3545"
    
    venue_name = row.get("venue_name", "N/A")
    city = row.get("city", "N/A")
    country = row.get("country", "N/A")
    vertical = row.get("vertical", "N/A")
    score = row.get("evaluation_score", "N/A")
    article_url = row.get("article_url", "#")
    article_title = row.get("article_title", "Voir l'article")
    audit_reason = row.get("audit_opportunity_reason", "N/A")
    validated_by = row.get("validated_by", "N/A")
    comment = row.get("validation_comment", "-")
    
    html = f"""
    <html>
    <body style="font-family: Arial, sans-serif; max-width: 600px; margin: 0 auto;">
        <div style="background: {status_color}; color: white; padding: 20px; text-align: center;">
            <h1 style="margin: 0;">Opportunité {status}</h1>
        </div>
        
        <div style="padding: 20px; background: #f8f9fa;">
            <h2 style="color: #333; margin-top: 0;">{venue_name}</h2>
            <p><strong>Lieu:</strong> {city}, {country}</p>
            <p><strong>Verticale:</strong> {vertical}</p>
            <p><strong>Score IA:</strong> {score}/100</p>
        </div>
        
        <div style="padding: 20px;">
            <h3>Article</h3>
            <p><a href="{article_url}">{article_title}</a></p>
            
            <h3>Justification IA</h3>
            <p style="background: #e9ecef; padding: 10px; border-radius: 5px;">
                {audit_reason}
            </p>
        </div>
        
        <div style="padding: 20px; background: #f8f9fa; border-top: 1px solid #ddd;">
            <p><strong>Validé par:</strong> {validated_by}</p>
            <p><strong>Commentaire:</strong> {comment}</p>
        </div>
        
        <div style="padding: 10px; text-align: center; color: #666; font-size: 12px;">
            <p>Weak Signals Pipeline - L-Acoustics</p>
        </div>
    </body>
    </html>
    """
    return html

print("✓ Fonction format_validation_email chargée")

In [ ]:
for idx, row in df_pending_emails.iterrows():
    subject = f"[Weak Signals] Opportunité validée: {row['venue_name']}"
    body = format_validation_email(row)
    to_email = row['saler_email']
    
    print(f"Envoi email à {to_email}...")
    
    payload = {
        "executionData": {
            "parameters": {
                "to_email": to_email,
                "subject": subject,
                "body": body
            }
        }
    }
    
    response = requests.post(
        f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/items/{pipeline_id}/jobs/instances?jobType=Pipeline",
        headers=headers,
        json=payload
    )
    
    if response.status_code in [200, 201, 202]:
        print(f"✓ Pipeline déclenché pour {to_email}")
    else:
        print(f"✗ Erreur: {response.status_code} - {response.text}")

In [ ]:
# Supprimer la table existante pour la recréer avec le nouveau schema
spark.sql("DROP TABLE IF EXISTS landing_validations")